In [1]:
import pandas as pd

amazon_df = pd.read_csv('Amazon_Reviews.csv')
amazon_df.head(10)

,Review,Label
0,Stuning even for the non-gamer: This sound tr...,1
1,The best soundtrack ever to anything.: I'm re...,1
2,Amazing!: This soundtrack is my favorite musi...,1
3,Excellent Soundtrack: I truly like this sound...,1
4,"Remember, Pull Your Jaw Off The Floor After H...",1
5,an absolute masterpiece: I am quite sure any ...,1
6,"Buyer beware: This is a self-published book, ...",0
7,Glorious story: I loved Whisper of the wicked...,1
8,A FIVE STAR BOOK: I just finished reading Whi...,1
9,Whispers of the Wicked Saints: This was a eas...,1


In [2]:
from nltk.tokenize import word_tokenize

y = amazon_df['Label']
amazon_df.drop(columns='Label', inplace=True)

In [3]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(amazon_df,y,test_size = 0.2, random_state=42)

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer

tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocesing(Review):
    final_tokens = ' '
    tokens = tokenizer.tokenize(Review)
    
    pure_tokens = [token.lower() for token in tokens if token.lower() not in stopwords.words('english')]
    stemmed_tokens = [stemmer.stem(pure_token) for pure_token in pure_tokens]
    final_tokens = final_tokens.join(stemmed_tokens)
    
    return final_tokens

x_train['final_Review'] = x_train['Review'].apply(preprocesing)

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', use_idf=True)

vectorizer.fit(x_train['final_Review'])
x_test['final_Review'] = x_test['Review'].apply(preprocesing)
x_train_Tfidf = vectorizer.transform(x_train['final_Review'])
x_test_Tfidf = vectorizer.transform(x_test['final_Review'])

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(x_train_Tfidf.toarray(), y_train)

y_pred = clf.predict(x_test_Tfidf.toarray())
y_pred

array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0], dtype=int64)

In [8]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[ 8,  7],
       [ 1, 24]], dtype=int64)

In [9]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.53      0.67        15
           1       0.77      0.96      0.86        25

    accuracy                           0.80        40
   macro avg       0.83      0.75      0.76        40
weighted avg       0.82      0.80      0.79        40

